In [1]:
from jwst_SNR import jwst_SNRclass
import numpy as np
from nirspec_sn import *
%matplotlib notebook

In [2]:
from background4jwst import background4jwstclass
background4jwst=background4jwstclass()
    
# set the position for which the background will be calculated. set_position_by_target uses 
# pre-defined positions, set_position can be used to set any position
print(background4jwst.defaulttargets)
background4jwst.set_position_by_target('ElGordo')
# alternatively, background4jwst.set_position('01 02 55.2','-49 14 29.3','ElGordo')
# Get the background from pandeia for this position
# optional parameters: lam, thresh
background4jwst.calc_background()

{'ElGordo': ('01 02 55.2', '-49 14 29.3'), 'EmptyERS': ('03 32 42.397', '-27 42 7.93'), 'NEP-TDF': ('17:22:47.896', '+65:49:21.54'), 'NEP-DF': ('17:40:08.00', '+69:00:08.00'), 'CDF-S': ('03:32:28.0', '−27:48:30')}
Calculating background for position ElGordo (15.730000,-49.241472)


In [3]:
jwst_SNR=jwst_SNRclass(instrument='nirspec')
#jwst_SNR=jwst_SNRclass(instrument='nircam')

jwst_SNR.set_background4jwst(50,target='EmptyERS')

Initializing pandeia with nirspec, None
Initializing nirspec fixed_slit
Calculating background for position EmptyERS (53.176654,-27.702203)
### Calculating percentile: 50
day: 187


0

In [9]:
from astropy.io import ascii

file = './data/siebert_phase_binned_2day_N=99_Nspec=153_phase=p0.95_dm15=1.15_z=0.014.txt'
data = ascii.read(file)  
df = data.to_pandas()
snia = S.ArraySpectrum(df['Wavelength'].values,df['Flux'].values,waveunits='angstrom',
               fluxunits='flam')
snia = snia.redshift(4)

Warning, 26 of 4762 bins contained negative fluxes; they have been set to zero.


In [5]:
# this part fials
jwst_SNR.verbose=2
jwst_SNR.Imaging_texp_table(['F200W'],np.arange(24.0,30.0,1),5,spec=snia,wave=2,width=.5)
jwst_SNR.texp.write('NIRSpec_SNR4.txt',formatters=jwst_SNR.formatters4texptable)

#############################
#### Filter F200W, mag 24.00 for S/N=5 
#############################
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 16, 'NINT': 1, 'tint': 233.4, 'NEXP': 4, 'texp': 933.7}


total texp  992.0445199999999
filter:F200W mag:24.00, target exptime:1000.0  == SNR=13.20 exptime=992.0
SNR= 13.20 for starting texp= 992.0
texp guess: 142.3
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 4, 'NINT': 1, 'tint': 58.4, 'NEXP': 4, 'texp': 233.4}


total texp  291.77779999999996
filter:F200W mag:24.00, target exptime:233.4  == SNR=2.68 exptime=291.8
SNR=  2.68 for next texp= 291.8
SNR=  2.68<5.00 for texp= 291.8, checking the next larger exptime...
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 8, 'NINT': 1, 'tint': 116.7, 'NEXP': 4, 'texp': 466.8}


total texp  525.20004
filter:F200W mag:24.00, target exptime:466.8  == SNR=6.58 exptime=525.2
SNR=  6.58>=5.00 for texp= 525.2!! SUCCESS!
#############################
#### Filter F200W, mag 24.20 for S/N=5 
#############################
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 16, 'NINT': 1, 'tint': 233.4, 'NEXP': 4, 'texp': 933.7}


total texp  992.0445199999999
filter:F200W mag:24.20, target exptime:1000.0  == SNR=11.31 exptime=992.0
SNR= 11.31 for starting texp= 992.0
texp guess: 194.0
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 4, 'NINT': 1, 'tint': 58.4, 'NEXP': 4, 'texp': 233.4}


...

total texp  758.6222799999999
filter:F200W mag:24.80, target exptime:700.3  == SNR=5.24 exptime=758.6
SNR=  5.24>5.00 for texp= 758.6, checking the next lower exptime...
BLEH  700.3
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 12, 'NINT': 1, 'tint': 175.1, 'NEXP': 4, 'texp': 700.3}


total texp  758.6222799999999
filter:F200W mag:24.80, target exptime:700.3  == SNR=5.24 exptime=758.6
SNR=  5.24>5.00 for texp= 758.6, checking the next lower exptime...
BLEH  700.3
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 12, 'NINT': 1, 'tint': 175.1, 'NEXP': 4, 'texp': 700.3}


total texp  758.6222799999999
filter:F200W mag:24.80, target exptime:700.3  == SNR=5.24 exptime=758.6
SNR=  5.24>5.00 for texp= 758.6, checking the next lower exptime...
BLEH  700.3
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 12, 'NINT': 1, 'tint': 175.1, 'NEXP': 4, 'texp': 700.3}


total texp  758.6222799999999
filter:F200W mag:24.80, target exptime:700.3  == SNR=5.24 exptime=758.6
SNR=  5.24>5.00 for texp= 758.6, checking the next lower exptime...
BLEH  700.3
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 12, 'NINT': 1, 'tint': 175.1, 'NEXP': 4, 'texp': 700.3}


KeyboardInterrupt: 

In [ ]:
jwst_SNR.ETCresults

In [ ]:
jwst_SNR.ETCresults

In [ ]:
plt.figure()
plt.plot(snia.wave,snia.flux)

In [ ]:
jwst_SNR.verbose=2
jwst_SNR.Imaging_SNR_table(['F200W','F444W'],np.arange(25.0,26.0,0.5),1000,spec=snia,wave=2,width=.5)

In [ ]:
import math

In [ ]:
math.pow(5/2,2)

In [ ]:
res = jwst_SNR.ETCresults

In [ ]:
snr = res['1d']['sn']

In [ ]:
plt.figure()
plt.plot(snr[0],snr[1])

In [ ]:
plt.axvspan()